In [2]:
import torch
from torchvision import transforms, datasets, models
import torch.nn as nn
import numpy as np
import os
import time
import copy
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader



In [45]:
train_data_path = 'data/train'
val_data_path = 'data/validation'

# Define transforms for the training data and testing data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224), # rescale the image
        transforms.RandomHorizontalFlip(), # randomly flip image horizontally
        transforms.RandomRotation(10), # randomly rotate image
        transforms.ToTensor(), # convert the image to a Tensor
        transforms.Normalize([0.485, 0.456, 0.406], # normalize image for pre-trained model
                                [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize(256), # rescale the image
        transforms.CenterCrop(224), # crop image
        transforms.ToTensor(), # convert the image to a Tensor
        transforms.Normalize([0.485, 0.456, 0.406], # normalize image for pre-trained model
                                [0.229, 0.224, 0.225])
    ]),
}

train_dataset = datasets.ImageFolder(train_data_path, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_data_path, transform=data_transforms['validation'])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 32, shuffle=True)
# Dataloader iterators, make sure to shuffle

    

In [54]:
model = models.resnet50(pretrained=True)
# Freeze model weights
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                    nn.ReLU(),
                                    nn.Dropout(0.2),
                                    nn.Linear(512, 2),
                                    nn.LogSoftmax(dim=1))


# If GPU is available, use GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define Optimizer and Loss Function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)


c:\Users\zhuoq\anaconda3\envs\tensorflow2.6.0\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\zhuoq\anaconda3\envs\tensorflow2.6.0\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [56]:
# Train the model
epochs = 20
for epoch in range(epochs):
    train_loss = 0.0
    val_loss = 0.0
    accuracy = 0.0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # Move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        # Clear the gradients
        optimizer.zero_grad()
        # Forward pass
        output = model(data)
        # Calculate the batch loss
        loss = criterion(output, target)
        # Backward pass
        loss.backward()
        # Perform optimization
        optimizer.step()
        # Update train loss
        train_loss += loss.item()*data.size(0)
    # Calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    # Print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))
    # Save model
    torch.save(model.state_dict(), 'model.pt')
    
    # Validation
    model.eval()
    for batch_idx, (data, target) in enumerate(val_loader):
        # Move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        # Forward pass
        output = model(data)
        # Calculate the batch loss
        loss = criterion(output, target)
        # Update average validation loss 
        val_loss += loss.item()*data.size(0)
        # Calculate validation accuracy
        _, pred = torch.max(output, dim=1)
        correct_tensor = pred.eq(target.data.view_as(pred))
        accuracy += torch.mean(correct_tensor.type(torch.FloatTensor))
    # Calculate average losses
    val_loss = val_loss/len(val_loader.dataset)
    accuracy = accuracy/len(val_loader)
    # Print training/validation statistics 
    print('Epoch: {} \tValidation Loss: {:.6f} \tAccuracy: {:.2f}%'.format(
        epoch+1, 
        val_loss,
        accuracy*100
        ))
    # Save model
    torch.save(model.state_dict(), 'model.pt')

Epoch: 1 	Training Loss: 0.339924
Epoch: 1 	Validation Loss: 0.235196 	Accuracy: 90.33%
Epoch: 2 	Training Loss: 0.310929
Epoch: 2 	Validation Loss: 0.200369 	Accuracy: 91.47%
Epoch: 3 	Training Loss: 0.300214
Epoch: 3 	Validation Loss: 0.197001 	Accuracy: 91.82%
Epoch: 4 	Training Loss: 0.289212
Epoch: 4 	Validation Loss: 0.220513 	Accuracy: 91.47%
Epoch: 5 	Training Loss: 0.272825
Epoch: 5 	Validation Loss: 0.173659 	Accuracy: 92.51%
Epoch: 6 	Training Loss: 0.286220
Epoch: 6 	Validation Loss: 0.188989 	Accuracy: 92.71%
Epoch: 7 	Training Loss: 0.285680
Epoch: 7 	Validation Loss: 0.169770 	Accuracy: 93.45%
Epoch: 8 	Training Loss: 0.266563
Epoch: 8 	Validation Loss: 0.182554 	Accuracy: 92.26%
Epoch: 9 	Training Loss: 0.254443
Epoch: 9 	Validation Loss: 0.158547 	Accuracy: 94.15%
Epoch: 10 	Training Loss: 0.259015
Epoch: 10 	Validation Loss: 0.149539 	Accuracy: 93.55%
Epoch: 11 	Training Loss: 0.260246
Epoch: 11 	Validation Loss: 0.155111 	Accuracy: 93.25%
Epoch: 12 	Training Loss: 0.

In [73]:
# Test the model
test_data_path = 'data/test'
test_dataset = datasets.ImageFolder(test_data_path, transform=data_transforms['validation'])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32, shuffle=True)
model.state_dict(torch.load('model.pt'))
model.eval()
accuracy = 0.0
error_count = 0
for batch_idx, (data, target) in enumerate(test_loader):
    data, target = data.to(device), target.to(device)
    output = model(data)
    _, pred = torch.max(output, dim=1)
    #添加图像名称，预测结果，真实结果
    for i in range(len(target)):
        print(test_loader.dataset.imgs[i][0].split('/')[-1],
              "Pre Res:","damage" if pred[i] == 1 else "no_damage", 
              "Actul Res:","damage" if target[i] == 1 else "no_damage")
        if pred[i] != target[i]:
            error_count += 1
            print("Error Prediction")
    
    correct_tensor = pred.eq(target.data.view_as(pred))
    accuracy += torch.mean(correct_tensor.type(torch.FloatTensor))
accuracy = accuracy/len(test_loader)
error_rate = error_count/len(test_loader.dataset)
print('Test Accuracy: {:.2f}%'.format(accuracy*100), "Error Rate: {:.2f}%".format(error_rate*100))


c:\Users\zhuoq\anaconda3\envs\tensorflow2.6.0\lib\site-packages\torch\nn\modules\module.py:1802: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


test\damage\-93.548123_30.900623.jpeg Pre Res: no_damage Actul Res: no_damage
test\damage\-93.560128_30.894917.jpeg Pre Res: damage Actul Res: damage
test\damage\-93.578271_30.779923999999998.jpeg Pre Res: damage Actul Res: damage
test\damage\-93.590598_30.694956.jpeg Pre Res: no_damage Actul Res: no_damage
test\damage\-93.604017_30.793719.jpeg Pre Res: no_damage Actul Res: no_damage
test\damage\-93.6141_30.754263.jpeg Pre Res: no_damage Actul Res: no_damage
test\damage\-93.618483_30.730039.jpeg Pre Res: no_damage Actul Res: no_damage
test\damage\-93.65613_30.076164000000002.jpeg Pre Res: damage Actul Res: damage
test\damage\-93.658785_30.206929.jpeg Pre Res: damage Actul Res: damage
test\damage\-93.659161_30.202384999999996.jpeg Pre Res: no_damage Actul Res: no_damage
test\damage\-93.65936500000001_30.138793.jpeg Pre Res: no_damage Actul Res: no_damage
test\damage\-93.659805_30.138094.jpeg Pre Res: no_damage Actul Res: no_damage
test\damage\-93.660431_30.20485.jpeg Pre Res: damage Act